# Финальный проект по курсу Data Science
![](https://gipermed.ru/upload/iblock/e3d/e3d6730be6efe6548610fc6d500c1e03.jpg)

Реальный ноутбук к соревнованию **"Google Brain - Ventilator Pressure Prediction"**

## Описание проекта
Суть задачи заключается  в предсказании необходимого давления, создаваемого аппаратом для искусственной вентиляции лёгких, подключённого к находящемуся без сознания пациенту. Симуляция такого аппарата намного дешевле и безопаснее реальных клинических испытаний. В итоге алгоритм поможет в разработке аппаратов, более гибко адаптирующихся к конкретному пациенту. 

В данном ноутбуке будут рассмотрены различные подходы к решению данной задачи, а также указаны результаты реальных сабмитов на соревнование. 

# 1. Подготовка к работе
## 1.1 Импорт необходимых библиотек

In [ ]:
import numpy as np  
import pandas as pd  

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import VotingRegressor

from catboost import CatBoostRegressor

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data

import gc

In [ ]:
#Зафиксируем random seed для воспроизводимости экспериментов
random_seed = 42

## 1.2 Определение функций

In [ ]:
#Анализ датасетов на отсутствующие данные
def missing_values_table(df):
        # Всего пустых позиций
        mis_val = df.isnull().sum()
        
        # Процент пустых позиций
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Таблица с результатами
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Переименование колонок в таблице
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Сортировка таблицы по убыванию % отсутствующих данных
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)
        
        # Итоговая информация в виде текста
        print ("В изучаемом датасете " + str(df.shape[1]) + " столбцов.\n"      
            "Всего " + str(mis_val_table_ren_columns.shape[0]) +
              " столбцов содержат отсутствующие значения.")
        
        # Таблица с результатами
        return mis_val_table_ren_columns

In [ ]:
#Анализ датасетов на уникальные данные
def unique_data(df, col):
    unique_d = pd.DataFrame(columns=['Parameter', 'unique'])
    for i in range(len(col)):
        unique_d.loc[i] = [col[i], df[col[i]].nunique(dropna = True)]
    return unique_d

In [ ]:
#Поскольку в ноутбуке будет много разных моделей, построим функцию,составляющую статистическую таблицу результатов
def cumulated_res(data, model,description, sub):
    l = len(data)
    data.loc[l]= [model, description, sub]
    return data
#Датафрейм для добавления результатов
df_cum = pd.DataFrame(columns=['Модель', 'Описание', 'Submission'])

In [ ]:
#Функция статистики
def statistic(df, col):
    median = df[col].median()
    IQR = df[col].quantile(0.75) - df[col].quantile(0.25)
    perc25 = df[col].quantile(0.25)
    perc75 = df[col].quantile(0.75)
    l=perc25 - 1.5*IQR 
    r=perc75 + 1.5*IQR
    print("Для {0} IQR: {1}, ".format(col,IQR),"Границы выбросов: [{0}, {1}].".format(l, r))
    print('Всего {} выбросов'.format(df[df[col] > r][col].count()+df[df[col] < l][col].count()))

In [ ]:
#Функция гистограммы и ящика с усами
def graph_num(df, col, size = 6):
    fig, (g1, g2) = plt.subplots(1, 2, figsize = (2*size,size))
    fig.suptitle('Histogram and boxplot for {0} '.format(col), fontsize=20)
    g1.hist(df[col], bins = 20, histtype = 'bar', align = 'mid', rwidth = 0.8, color = 'blue') # гистограмма
    g2.boxplot(df[col], vert = False)  # выбросы
    plt.figtext(0.5, 0, col, fontsize = 16)
    plt.show

In [ ]:
#Функция гистограммы для одного временного ряда, параметр по выбору
def graph_ts(df,param, value):
    df_ts = df.loc[df['breath_id'] == value]
    sns.histplot(df_ts[param], kde=True)

# 2. Импорт и обзор данных
Данные представлены в виде двух выложенных на Kaggle тестового и тренировочного датасетов и шаблона для сабмитов. Известно, что в данных содержится следующая информация:

**id** - Уникальный идентификатор каждого вдоха.   
  **breath_id -** Уникальный идентификатор серии вдохов. Известно, что данные представлены в виде набора временных рядов, состоящих из одинакового количества вдохов.
  **R** - параметр лёгких, показывающий органичения дыхания (в cmH2O/L/S). Физически это изменение давления в зависимости от изменения в потоке воздуха  (объём воздуха за определённое время). Интуитивно это можно представитькак попытку надуть шарик через соломинку. Параметр R можно изменить, меняя диаметр соломинки. Чем больше R, тем сложнее надувать.   
  **C** - параметр лёгких, отражающий их "податливость"(в mL/cmH2O). Физически это изменения в объёме при изменении давления. Интуитивно это тот же самый пример с шариком. С можно изменить, изменяя толщину материала шарика. Чем выше С, тем тоньше стенка и тем легче надувать шарик.   
  **time_step -** реальное время  
  **u_in -** параметр входа на клапане от 0 (полностью закрыт, воздух не вдыхается) до 100.  
  **u_out -** Параметр выхода на клапане. 0 клапан закрыт, пациент вдыхает или 1 - клапан открыт, пациент выдыхает.  
  **pressure -** давление воздуха в cmH2O - целевой параметр.

In [ ]:
#Выгрузим датасеты
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

print(train.shape, test.shape)

In [ ]:
submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')

## 2.1 Предварительный обзор
Применить pandas_profiling не удалось из-за большого размера датасетов. Каждый раз процедура намертво зависала. 

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
#Перечень заголовков столбцов для каждого датасета
columns_tr = list(train.columns)
columns_te = list(test.columns)
print("train", columns_tr)
print("test", columns_te)

In [ ]:
# Поиск пустых значений для тренировочного датасета
missing_values_table(train)

In [ ]:
# Поиск пустых значений для тестового датасета
missing_values_table(test)

In [ ]:
#Проверим, встречаются ли одинаковые серии вдохов в тестовом и тренировочном датасетах
train_breath_id = [x for x in (np.unique(train['breath_id']))]
test_breath_id = [x for x in (np.unique(test['breath_id']))]
set(test_breath_id).intersection(train_breath_id)

In [ ]:
# Поиск уникальных значений для тренировочного датасета
unique_data(train, columns_tr)

In [ ]:
# Поиск уникальных значений для тестового датасета
unique_data(test, columns_te)

In [ ]:
train['id'].nunique()/train['breath_id'].nunique()

In [ ]:
test['id'].nunique()/test['breath_id'].nunique()

In [ ]:
train.groupby("breath_id")["time_step"].count()

In [ ]:
test.groupby("breath_id")["time_step"].count()

Поскольку нам известно,  что серии вдохов имеют одинаковый размер, будем в дальнейшем в качестве количества вдохов в ряду использовать  80.
## 2.1 Посмотрим распределения и статистики

In [ ]:
for c in columns_tr:
    statistic(train, c)

In [ ]:
for c in columns_tr:
    graph_num(train, c)

In [ ]:
for c in columns_te:
    statistic(test, c)

In [ ]:
for c in columns_te:
    graph_num(test, c)

In [ ]:
for i in ['R','C','u_out']:
    print(i)
    print(train[i].value_counts())

Мы не видим ни одного нормального распределения. Также 3 параметра можно перевести в категориальные. 
Проверим корреляции.

In [ ]:
sns.heatmap(train.corr(), vmin = -1, vmax = +1, annot = True, cmap = 'coolwarm')

Более менее коррелируют только давление и параметр выхода на клапане, а также время выдоха. 
Посмотрим распределение давления в рамках одного дыхания.

In [ ]:
graph_ts(train,'pressure', 18)

In [ ]:
graph_ts(train, 'u_in',45008)

In [ ]:
graph_ts(train,'R', 45094)

In [ ]:
graph_ts(train, 'C', 18)

In [ ]:
graph_ts(train,'u_out', 2)

Не самые говорящие графики. Попробуем другие варианты. Рассмотрим в рамках одной серии из тестового датасета. 

In [ ]:
breath_id_1 = train[train['breath_id'] == 1]
breath_id_1.head()

In [ ]:
fig, ax1 = plt.subplots(figsize = (6, 4))
ax2 = ax1.twinx()
ax1.plot(breath_id_1['time_step'], breath_id_1['pressure'], 'm-', label='pressure')
ax1.plot(breath_id_1['time_step'], breath_id_1['u_in'], 'g-', label='u_in')
ax2.plot(breath_id_1['time_step'], breath_id_1['u_out'], 'b-', label='u_out')

ax1.set_xlabel('Timestep')

R = breath_id_1['R'][0]
C = breath_id_1['C'][0]
ax1.set_title(f'breath_id:{1}, R:{R}, C:{C}')

ax1.legend(loc=(1.1, 0.8))
ax2.legend(loc=(1.1, 0.7))
plt.show()

In [ ]:
sns.lineplot(x = 'id',y='pressure',data=breath_id_1[breath_id_1['u_out']==0],color='green',label='inhale pressure');
sns.lineplot(x = 'id',y='pressure',data=breath_id_1[breath_id_1['u_out']==1],color='orange',label='exhale pressure');
sns.lineplot(x = 'id',y='u_in',data=breath_id_1,color='blue',label='valve pressure')
plt.title(f"Variation of Pressure and Input valve position during breath Id 1");
plt.show()

Учитывая размер датасетов, будем сразу удалять ненужные списки и датафреймы и оптимизировать память. 

In [ ]:
del breath_id_1
del columns_tr
del columns_te
del train_breath_id
del test_breath_id

gc.collect()

Начнём собирать модели.
Для начала отработаем на действующих признаках, не переводя их во временные ряды, затем сделаем преобразование имеющихся признаков, сформировав новые, затем добавим временные ряды. Что касается моделей, то в данном случае можно попробовать регрессии, ансамбли, нейросеть и поблендить результаты.   Для того, чтобы не выгружать заново датасеты, датафреймы будут преобразовываться из исходных train и test.
# 3. Модели
# 3.1 Регрессия на имеющихся признаках
Ничего не будем преобразовывать

In [ ]:
X_n=train.drop(['pressure','id'], axis=1)
y_n=train['pressure']
id_test = test['id']
X_n_t = test.drop(['id'], axis=1)

In [ ]:
X_n_train, X_n_test, y_n_train, y_n_test = train_test_split(X_n,y_n, test_size=0.2, random_state=random_seed)

In [ ]:
catboost=CatBoostRegressor()
grid={'depth': [6,8,10],
      'learning_rate' : [0.01, 0.05, 0.1],
      'iterations'    : [30, 50, 100]}

In [ ]:
grid_catboost=GridSearchCV(estimator=catboost, param_grid = grid, cv = 2, n_jobs=-1)

In [ ]:
grid_catboost.fit(X_n_train,y_n_train)

In [ ]:
preds_cb = grid_catboost.predict(X_n_t)

In [ ]:
preds_cb

In [ ]:
sub_cb = submission
sub_cb['pressure'] = preds_cb
sub_cb.to_csv('submission_cb.csv', index=False)

In [ ]:
cumulated_res(df_cum, 'CatBoost', 'без EDA', '4.2000')

In [ ]:
del preds_cb
del X_n
del X_n_train
del X_n_test
del y_n
del y_n_train
del y_n_test
del X_n_t
del id_test
gc.collect()

# 3.2 Регрессия на "наивных" признаках

Преобразуем некоторые признаки в категориальные.

In [ ]:
#"Наивные" датасеты
df_tr_naiv = train
df_te_naiv = test
type(df_tr_naiv)

In [ ]:
df_tr_naiv['R'] = df_tr_naiv['R'].astype(str)
df_tr_naiv['C'] = df_tr_naiv['C'].astype(str)
df_te_naiv['R'] = df_te_naiv['R'].astype(str)
df_te_naiv['C'] = df_te_naiv['C'].astype(str)
df_tr_naiv = pd.get_dummies(df_tr_naiv)
df_te_naiv = pd.get_dummies(df_te_naiv)
df_tr_naiv.info()

In [ ]:
df_te_naiv.info()

In [ ]:
y_naiv = df_tr_naiv['pressure']
id_test = df_te_naiv['id']

In [ ]:
X_naiv = df_tr_naiv
X_naiv.drop(['id', 'pressure', 'breath_id'], axis=1, inplace=True)
X_s_naiv = df_te_naiv
X_s_naiv.drop(['id', 'breath_id'], axis=1, inplace=True)

In [ ]:
X_naiv.head()

In [ ]:
X_naiv.info()

In [ ]:
X_s_naiv.info()

In [ ]:
del df_tr_naiv
del df_te_naiv
del id_test

gc.collect()

In [ ]:
#Обозначим модели
dtr = DecisionTreeRegressor(random_state=random_seed)
abr = AdaBoostRegressor(random_state=random_seed)
br = BaggingRegressor(random_state=random_seed)
ensemble =  VotingRegressor([('br', br), ('abr', abr), ('dtr', dtr)])

In [ ]:
test_preds_dtr = []
test_preds_abr = []
test_preds_br = []
test_preds_ens = []

Расчёты будут идти на 5 фолдах

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=random_seed)

In [ ]:
print('DecisionTreeRegressor')
for fold, (train_idx, test_idx) in enumerate(kf.split(X_naiv, y_naiv)):
    
    print('fold->',fold+1)
    X_naiv_train, X_Naivtest = X_naiv.loc[train_idx], X_naiv.loc[test_idx]
    y_naiv_train, y_naiv_test = y_naiv.loc[train_idx], y_naiv.loc[test_idx]
    dtr.fit(X_naiv_train, y_naiv_train)
    test_preds_dtr.append(dtr.predict(X_s_naiv))

In [ ]:
sub_dtr = submission
sub_dtr['pressure'] = sum(test_preds_dtr)/5
sub_dtr.to_csv('submission_dtr.csv', index=False)

In [ ]:
cumulated_res(df_cum, 'DecisionTreeRegressor', 'наивный feature engineering', '4.1192')

In [ ]:
del test_preds_dtr
gc.collect()

In [ ]:
print('AdaBoostRegressor')
for fold, (train_idx, test_idx) in enumerate(kf.split(X_naiv, y_naiv)):
    
    print('fold->',fold+1)
    X_naiv_train, X_Naivtest = X_naiv.loc[train_idx], X_naiv.loc[test_idx]
    y_naiv_train, y_naiv_test = y_naiv.loc[train_idx], y_naiv.loc[test_idx]
    abr.fit(X_naiv_train, y_naiv_train)
    test_preds_abr.append(abr.predict(X_s_naiv))

In [ ]:
sub_abr = submission
sub_abr['pressure'] = sum(test_preds_abr)/5
sub_abr.to_csv('submission_abr.csv', index=False)

In [ ]:
cumulated_res(df_cum, 'AdaBoostRegressor', 'наивный feature engineering', '6.9884')

In [ ]:
del test_preds_abr
gc.collect()

In [ ]:
print('BaggingRegressor')
for fold, (train_idx, test_idx) in enumerate(kf.split(X_naiv, y_naiv)):
    print('fold->',fold+1)
    X_naiv_train, X_Naivtest = X_naiv.loc[train_idx], X_naiv.loc[test_idx]
    y_naiv_train, y_naiv_test = y_naiv.loc[train_idx], y_naiv.loc[test_idx]
    br.fit(X_naiv_train, y_naiv_train)
    test_preds_br.append(br.predict(X_s_naiv))

In [ ]:
sub_br = submission
sub_br['pressure'] = sum(test_preds_br)/5
sub_br.to_csv('submission_br.csv', index=False)

In [ ]:
cumulated_res(df_cum, 'BaggingRegressor', 'наивный feature engineering', '3.7660')

In [ ]:
del test_preds_br
gc.collect()

In [ ]:
print('VotingRegressor')
for fold, (train_idx, test_idx) in enumerate(kf.split(X_naiv, y_naiv)):
    print('fold->',fold+1)
    X_naiv_train, X_Naivtest = X_naiv.loc[train_idx], X_naiv.loc[test_idx]
    y_naiv_train, y_naiv_test = y_naiv.loc[train_idx], y_naiv.loc[test_idx]
    ensemble.fit(X_naiv_train, y_naiv_train)
    test_preds_ens.append(ensemble.predict(X_s_naiv))

In [ ]:
del X_naiv
del y_naiv
del X_s_naiv

gc.collect()

In [ ]:
sub_ens = submission
sub_ens['pressure'] = sum(test_preds_ens)/5
sub_ens.to_csv('submission_ens.csv', index = False)

In [ ]:
cumulated_res(df_cum, 'VotingRegressor', 'наивный feature engineering', '4.5196')

In [ ]:
del test_preds_ens
gc.collect()

In [ ]:
sub_blend = submission
sub_blend['pressure'] = (sub_dtr['pressure'],+sub_abr['pressure']+ sub_br['pressure']+sub_ens['pressure']+ sub_cb['pressure'])/5
sub_blend.to_csv('submission_blend.csv', index=False)

In [ ]:
del sub_cb
del sub_dtr
del sub_abr
del sub_br
del sub_ens
del sub_blend

gc.collect()

# 3.3 Регрессия на доработанных признаках
Добавим некоторые признаки. 

Создадим датасеты для преобразования

In [ ]:
df_tr = train
df_te = test

In [ ]:
del train
del test

gc.collect()

In [ ]:
#Последнее значение на выходном клапане last_value_u_in
idxmax_time_step = df_tr.groupby('breath_id')['time_step'].idxmax()
last_value_u_in = df_tr.loc[idxmax_time_step, ['breath_id','u_in']]
last_value_u_in.columns = ['breath_id','last_value_u_in']

df_tr = df_tr.merge(last_value_u_in, on='breath_id')
 
idxmax_time_step = df_te.groupby('breath_id')['time_step'].idxmax()
last_value_u_in = df_te.loc[idxmax_time_step, ['breath_id','u_in']]
last_value_u_in.columns = ['breath_id','last_value_u_in']

df_te = df_te.merge(last_value_u_in, on='breath_id')
df_te.head()

In [ ]:
df_tr.head()

In [ ]:
del idxmax_time_step
del last_value_u_in
gc.collect()

In [ ]:
#Среднее значение на входном клапане mean value u_in
mean_u_in = df_tr.groupby('breath_id')['u_in'].mean().to_frame()
mean_u_in.columns = ['mean_value_u_in']
df_tr = df_tr.merge(mean_u_in,on='breath_id')

mean_u_in = df_te.groupby('breath_id')['u_in'].mean().to_frame()
mean_u_in.columns = ['mean_value_u_in']
df_te = df_te.merge(mean_u_in,on='breath_id')
df_te.head()

In [ ]:
#Разница на входном клапане diff of value_u_in
df_tr['diff_u_in'] = df_tr.groupby('breath_id')['u_in'].diff()
df_tr = df_tr.fillna(0)
df_tr.head()

df_te['diff_u_in'] = df_te.groupby('breath_id')['u_in'].diff()
df_te = df_te.fillna(0)

In [ ]:
df_te.head()

In [ ]:
#Сумма на входном клапане накопительным итогом
df_tr['u_in_cumsum'] = (df_tr['u_in']).groupby(df_tr['breath_id']).cumsum()
df_te['u_in_cumsum'] = (df_te['u_in']).groupby(df_te['breath_id']).cumsum()

In [ ]:
#Сумма на входном клапане обычная
sum_u_in = df_tr.groupby('breath_id')['u_in'].sum().to_frame()
sum_u_in.columns = ['sum_value_u_in']
df_tr = df_tr.merge(sum_u_in,on='breath_id')

sum_u_in = df_te.groupby('breath_id')['u_in'].sum().to_frame()
sum_u_in.columns = ['sum_value_u_in']
df_te = df_te.merge(sum_u_in,on='breath_id')

In [ ]:
del sum_u_in
gc.collect()

In [ ]:
#Отношение накопительной суммы к обычной на входном клапане
df_tr["u_in_cumsum_rate"] = df_tr["u_in_cumsum"] / df_tr["sum_value_u_in"]
df_te["u_in_cumsum_rate"] = df_te["u_in_cumsum"] / df_te["sum_value_u_in"]

In [ ]:
#Если у вдоха нулевое значение на входном клапане
df_tr[df_tr["sum_value_u_in"] == 0]


In [ ]:
df_te[df_te["sum_value_u_in"] == 0]

In [ ]:
df_tr = df_tr.fillna(0)
df_te = df_te.fillna(0)

In [ ]:
# Данные входного клапана при сдвиге на один шаг
df_tr['lag_u_in'] = df_tr.groupby('breath_id')['u_in'].shift(1)
df_tr = df_tr.fillna(0)

In [ ]:
df_te['lag_u_in'] = df_te.groupby('breath_id')['u_in'].shift(1)
df_te = df_te.fillna(0)

In [ ]:
# Данные входного клапана при сдвиге на два шага
df_tr['lag_2_u_in'] = df_tr.groupby('breath_id')['u_in'].shift(2)
df_tr = df_tr.fillna(0)

In [ ]:
df_te['lag_2_u_in'] = df_te.groupby('breath_id')['u_in'].shift(2)
df_te = df_te.fillna(0)

In [ ]:
# Данные входного клапана при сдвиге на минус один шаг
df_tr['lag_-1_u_in'] = df_tr.groupby('breath_id')['u_in'].shift(-1)
df_tr = df_tr.fillna(0)
df_te['lag_-1_u_in'] = df_te.groupby('breath_id')['u_in'].shift(-1)
df_te = df_te.fillna(0)

In [ ]:
# Данные входного клапана при сдвиге на минус два шага
df_tr['lag_-2_u_in'] = df_tr.groupby('breath_id')['u_in'].shift(-2)
df_tr = df_tr.fillna(0)
df_te['lag_-2_u_in'] = df_te.groupby('breath_id')['u_in'].shift(-2)
df_te = df_te.fillna(0)

In [ ]:
# Данные входного клапана при сдвиге на минус три шага
df_tr['lag_-3_u_in'] = df_tr.groupby('breath_id')['u_in'].shift(-3)
df_tr = df_tr.fillna(0)
df_te['lag_-3_u_in'] = df_te.groupby('breath_id')['u_in'].shift(-3)
df_te = df_te.fillna(0)

In [ ]:
# Данные входного клапана при сдвиге на три шага
df_tr['lag_3_u_in'] = df_tr.groupby('breath_id')['u_in'].shift(3)
df_tr = df_tr.fillna(0)
df_te['lag_3_u_in'] = df_te.groupby('breath_id')['u_in'].shift(3)
df_te = df_te.fillna(0)

In [ ]:
# Максимум по вдоху
df_tr["max_u_in_breathid"] = df_tr.groupby("breath_id")["u_in"].transform("max")
df_te["max_u_in_breathid"] = df_te.groupby("breath_id")["u_in"].transform("max")

In [ ]:
## Минимум по вдоху
df_tr['breath_id__u_in__min'] = df_tr.groupby(['breath_id'])['u_in'].transform('min')
df_te['breath_id__u_in__min'] = df_te.groupby(['breath_id'])['u_in'].transform('min')

In [ ]:
#Произведение двух признаков
df_tr["R*C"] = df_tr['R'] * df_tr['C']
df_te['R*C'] = df_te['R'] * df_te['C']

In [ ]:
df_tr['breath_id__u_in__diffmax'] = df_tr.groupby(['breath_id'])['u_in'].transform('max') - df_tr['u_in']
df_tr['breath_id__u_in__diffmean'] = df_tr.groupby(['breath_id'])['u_in'].transform('mean') - df_tr['u_in']

df_te['breath_id__u_in__diffmax'] = df_te.groupby(['breath_id'])['u_in'].transform('max') - df_te['u_in']
df_te['breath_id__u_in__diffmean'] = df_te.groupby(['breath_id'])['u_in'].transform('mean') - df_te['u_in']

In [ ]:
df_tr['breath_id__u_in__diffmax'] = df_tr.groupby(['breath_id'])['u_in'].transform('max') - df_tr['u_in']
df_tr['breath_id__u_in__diffmean'] = df_tr.groupby(['breath_id'])['u_in'].transform('mean') - df_tr['u_in']

df_te['breath_id__u_in__diffmax'] = df_te.groupby(['breath_id'])['u_in'].transform('max') - df_te['u_in']
df_te['breath_id__u_in__diffmean'] = df_te.groupby(['breath_id'])['u_in'].transform('mean') - df_te['u_in']

In [ ]:
df_tr['area'] = df_tr['time_step'] * df_tr['u_in']
df_tr['area'] = df_tr.groupby('breath_id')['area'].cumsum()
df_te['area'] = df_te['time_step'] * df_te['u_in']
df_te['area'] = df_te.groupby('breath_id')['area'].cumsum()

Добавим категориальные переменные

In [ ]:
df_tr["train_test"] = "train"
df_te["train_test"] = "test"

In [ ]:
train_test_all = pd.concat([df_tr,df_te],axis=0)

In [ ]:
del df_tr
del df_te
gc.collect()

In [ ]:
train_test_all.head()

In [ ]:
train_test_all['R_C'] = [f'{r}_{c}' for r, c in zip(train_test_all['R'], train_test_all['C'])]

In [ ]:
train_test_all = pd.get_dummies(train_test_all,columns=["R_C"])

In [ ]:
train_test_all.columns

In [ ]:
train_test_all['time_diff']=train_test_all.time_step.diff().fillna(0)

In [ ]:
df_tr = train_test_all[train_test_all["train_test"] == "train"]
df_te = train_test_all[train_test_all["train_test"] == "test"]

In [ ]:
del train_test_all
gc.collect()

In [ ]:
LM = True
u_out_zero_only = False ## if train from only u_out=0 data 

In [ ]:
#train
if(u_out_zero_only):
    df_tr = df_tr[df_tr["u_out"] == 0]
    df_tr = df_tr.reset_index(drop=True)
X_train = df_tr.drop(["pressure","breath_id","train_test"],axis=1)
y_train = df_tr['pressure']
X_test = df_te.drop(["pressure","breath_id","train_test"],axis=1)

if(LM):
    scaler = StandardScaler()
    scaler.fit(X_train)
    #print(scaler.mean_)

    X_train_std = scaler.transform(X_train)


    lr = LinearRegression().fit(X_train_std, y_train)
    print("коэффициент детерминации = ",lr.score(X_train_std, y_train))


    #test
    X_test_std = scaler.transform(X_test)
    sub_lr=submission
    sub_lr['pressure'] = lr.predict(X_test_std)
    sub_lr.to_csv("submission_lm.csv",index=False)

In [ ]:
cumulated_res(df_cum, 'Linear Regression', 'feature engineering', '4.2873')

In [ ]:
df_cum.to_csv('cum.csv', index=False)

In [ ]:
del df_tr
del df_te
del X_train
del y_train
del X_test
del X_train_std
del X_test_std

gc.collect()

# 3.4 Нейросеть

Различные модели регрессии на данных, не разделённых на ряды по сету вдохов, дают не очень хороший результат. Попробуем прогнать данные через нейросеть. При этом подавать будем сетами по 80 шагов (на каждую серию вдохов).
Учитывая, что для целей очистки памяти датафреймы удалялись, заново выгрузим данные и сделаем дополнительные признаки, а также сделаем решейпинг. 

In [ ]:
train= pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test= pd.read_csv('../input/ventilator-pressure-prediction/test.csv')
test_ids= test['id'].to_numpy()

In [ ]:
# Обернём составление признаков в функцию
def preprocess(df):
    dfr= pd.get_dummies(df['R'], prefix= "R_")
    df= pd.concat([df, dfr], axis= 1)
    dfc= pd.get_dummies(df['C'], prefix= "C_")
    df= pd.concat([df, dfc], axis= 1)
    df= df.drop(['R', 'C'], axis= 1)

    df['u_in_cumsum']= df['u_in'].groupby(df['breath_id']).cumsum()
    df['time_step_cumsum']= df['time_step'].groupby(df['breath_id']).cumsum()
    
    df['u_in_min']= df['u_in'].groupby(df['breath_id']).transform('min')
    df['u_in_max']= df['u_in'].groupby(df['breath_id']).transform('max')
    df['u_in_mean']= df['u_in'].groupby(df['breath_id']).transform('mean')
   
    df['u_in_lag2']= df['u_in'].groupby(df['breath_id']).shift(2)
    df['u_in_lag1']= df['u_in'].groupby(df['breath_id']).shift(1)
    df['u_in_lag-1']= df['u_in'].groupby(df['breath_id']).shift(-1)
    df['u_in_lag-2']= df['u_in'].groupby(df['breath_id']).shift(-2)
    df= df.fillna(0)

    df['u_in_diff1']= df['u_in']- df['u_in_lag1']
    df['u_in_diff2']= df['u_in']- df['u_in_lag2']
    df['u_in_diff3']= df['u_in_max']- df['u_in']
    df['u_in_diff4']= df['u_in_mean']- df['u_in']

    df1= df[df['u_out'] == 0]
    df['mean_inspiratory_uin']= df1['u_in'].groupby(df['breath_id']).transform('mean')

    df2= df[df['u_out'] == 1]
    df['mean_expiratory_uin']= df2['u_in'].groupby(df['breath_id']).transform('mean')
    
    df['u_in_diff5']= df['mean_inspiratory_uin']- df['u_in']
    df['u_in_diff6']= df['mean_expiratory_uin']- df['u_in']
    
    df= df.fillna(0)
    
    df['delta_t']= df.groupby('breath_id')['time_step'].diff().fillna(0)
    df['delta_uin']= df.groupby('breath_id')['u_in'].diff().fillna(0)
    
    df['area']= df['u_in']*df['delta_t']
    df['area']= df.groupby('breath_id')['area'].cumsum()
    df['slope']= (df['delta_uin']/df['delta_t']).fillna(0)

    return df

In [ ]:
groups= train.breath_id.values.reshape(-1, 80)[:, 0]
groups.shape

train= preprocess(train)
targets= train['pressure'].to_numpy().reshape(-1, 80)
train.drop(['id','pressure', "breath_id"], axis= 1, inplace= True)

test= preprocess(test)
test.drop(['id', "breath_id"], axis= 1, inplace= True)
y_test= np.zeros(test.shape[0]).reshape(-1, 80)

In [ ]:
RS = RobustScaler()
train = RS.fit_transform(train)
test  = RS.transform(test)

num_features= train.shape[-1]
train= train.reshape(-1, 80, num_features)
test= test.reshape(-1, 80, num_features)

In [ ]:
class CustomDataset:
    def __init__(self, data, target):
        self.data= data
        self.target= target
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        current_sample= self.data[idx, :, :]
        current_target= self.target[idx, :]
        
        return torch.tensor(current_sample, dtype= torch.float), torch.tensor(current_target, dtype= torch.float)

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(RNNModel, self).__init__()
        
        hidden_dim= [400, 300, 200, 100]
        self.bilstm1= nn.LSTM(input_dim, hidden_dim[0], batch_first= True, bidirectional= True)
        self.norm1= nn.LayerNorm(hidden_dim[0]*2)
        
        self.bilstm2= nn.LSTM(hidden_dim[0]*2, hidden_dim[1], batch_first= True, bidirectional= True)
        self.norm2= nn.LayerNorm(hidden_dim[1]*2)
        
        self.bilstm3= nn.LSTM(hidden_dim[1]*2, hidden_dim[2], batch_first= True, bidirectional= True)
        self.norm3= nn.LayerNorm(hidden_dim[2]*2)
        
        self.bilstm4= nn.LSTM(hidden_dim[2]*2, hidden_dim[3], batch_first= True, bidirectional= True)
        self.norm4= nn.LayerNorm(hidden_dim[3]*2)
        
        self.fc1= nn.Linear(hidden_dim[3]*2, 100)
        self.fc2= nn.Linear(100, output_dim)

        
    def forward(self, X):
        pred, _= self.bilstm1(X)
        pred= self.norm1(pred)
        
        pred, _= self.bilstm2(pred)
        pred= self.norm2(pred)
        
        pred, _= self.bilstm3(pred)
        pred= self.norm3(pred)
        
        pred, _= self.bilstm4(pred)
        pred= self.norm4(pred)
        
        pred= self.fc1(pred)
        pred= F.selu(pred)
        
        pred= self.fc2(pred)
        pred= pred.squeeze(dim= 2)
        return pred

In [ ]:
def initialize_parameters(m):
    if isinstance(m, nn.LSTM):
        nn.init.orthogonal_(m.weight_ih_l0.data, gain= nn.init.calculate_gain('tanh'))
        nn.init.orthogonal_(m.weight_hh_l0.data, gain= nn.init.calculate_gain('tanh'))
        nn.init.orthogonal_(m.weight_ih_l0_reverse.data, gain= nn.init.calculate_gain('tanh'))
        nn.init.orthogonal_(m.weight_hh_l0_reverse.data, gain= nn.init.calculate_gain('tanh'))
        
        nn.init.constant_(m.bias_ih_l0.data, 0)
        nn.init.constant_(m.bias_hh_l0.data, 0)
        nn.init.constant_(m.bias_ih_l0_reverse.data, 0)
        nn.init.constant_(m.bias_hh_l0_reverse.data, 0)
        
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight.data)
        nn.init.constant_(m.bias.data, 0)

In [ ]:
#Опишем признаки модели
device= "cuda" if torch.cuda.is_available() else 'cpu'
INPUT_DIM= num_features
OUTPUT_DIM= 1
BATCH_SIZE= 1024

In [ ]:
def train_model(dataloader, model, criterion, optimizer):
    size= len(dataloader.dataset)
    model.train()
    batches= len(dataloader)
    train_loss= 0
    
    for batch_idx, (X, y) in enumerate(dataloader):
        X, y= X.to(device), y.to(device)

        scores= model(X)
        loss= criterion(scores, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        loss= loss.item()
        train_loss += loss
        
    train_loss_avg= train_loss/batches
    print(f"avg. train loss: {train_loss_avg}")
    return train_loss_avg

In [ ]:
def val_model(dataloader, model, criterion):
    
    size= len(dataloader.dataset)
    batches= len(dataloader)
    model.eval()
    test_loss= 0

    with torch.no_grad():
        for X, y in (dataloader):
            X, y= X.to(device), y.to(device)
      
            scores= model(X)
            test_loss += criterion(scores, y)

    test_loss /= batches
    print(f"avg test loss : {test_loss}")
    return test_loss

In [ ]:
def predict_model(dataloader, model):
    model.eval()
    y_pred= np.array([])
    
    with torch.no_grad():
        for X , y in dataloader:
            X, y= X.to(device), y.to(device)
            
            preds= model(X)
            preds= preds.flatten().cpu().numpy()
            
            y_pred= np.concatenate((y_pred, preds))
            
    return y_pred

In [ ]:
kfold= GroupKFold(n_splits= 5)
EPOCHS= 150
cv_scores= []
predictions= np.zeros(test_ids.shape[0])


for fold, (train_idx, val_idx) in enumerate(kfold.split(train, targets, groups= groups)):
    X_train, X_val= train[train_idx], train[val_idx]
    y_train, y_val= targets[train_idx], targets[val_idx]
    
    train_dataset= CustomDataset(data= X_train, target= y_train)
    val_dataset= CustomDataset(data= X_val, target= y_val)

    train_loader= data.DataLoader(train_dataset, batch_size= BATCH_SIZE)
    val_loader= data.DataLoader(val_dataset, batch_size= BATCH_SIZE)
    
    model= RNNModel(input_dim= INPUT_DIM, output_dim= OUTPUT_DIM).to(device)
    model.apply(initialize_parameters)

    criterion= nn.L1Loss()
    criterion.to(device)

    optimizer= optim.Adam(model.parameters(), lr= 0.001)
    scheduler= optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor= 0.5, patience= 10, verbose= True)
    
    best_valid_loss= float('inf')
    
    avg_train_losses= []
    avg_val_losses= []
    
    for t in range(EPOCHS):
        print(f"Epoch: {t+1}")
        train_loss= train_model(train_loader, model, criterion, optimizer)
        val_loss= val_model(val_loader, model, criterion)
        
        avg_train_losses.append(train_loss)
        avg_val_losses.append(val_loss)
        
        if (val_loss< best_valid_loss):
            best_valid_loss= val_loss
            ofilename = 'ventilator%d.pth' % fold
            torch.save(model.state_dict(),  ofilename)
        
        scheduler.step(val_loss)
    
    cv_scores.append(best_valid_loss)
    
    test_dataset= CustomDataset(data= test, target= y_test)
    test_loader= data.DataLoader(test_dataset, batch_size= BATCH_SIZE)
                       
    model.load_state_dict(torch.load('ventilator%d.pth' % fold, map_location=device))
    predictions += (predict_model(test_loader, model)/5)

In [ ]:
predictions

In [ ]:
cv_scores

In [ ]:
sub_nn= pd.DataFrame({'id': test_ids, 'pressure': predictions})
sub_nn.to_csv('submission.csv',index = False)

In [ ]:
cumulated_res(df_cum, 'LSTM', 'feature engineering', '0.2288')